In [ ]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *

In [ ]:
sc = SparkContext("local[*]", "Dataproc")
sqlContext = SQLContext(sc)

In [ ]:
#load data in spark
df_identity=sqlContext.read.option("header", "true").csv("data/train_identity.csv")
df_transaction=sqlContext.read.option("header", "true").csv("data/train_transaction.csv")

In [ ]:
# example de load d’un fichier plat (format csv)
#df_identity = sqlContext.read.load('data/train_identity.csv',
                                  #format='csv',
                                  #header='true')

In [ ]:
import pandas as pd
import numpy as np
import tensorflow_data_validation as tfdv

In [ ]:
# Méthode de préprocessing utilisés
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import lightgbm as lgb

In [ ]:
def df_type(df):
    return df.select_dtypes(include=['object']).columns, df.select_dtypes(exclude=['object']).columns

def quali_distribution(df, var_quali):
    trx = df[var_quali].value_counts().reset_index().rename({'index':'Value',var_quali:'Count'}, axis=1)
    trx['Share'] = np.round(trx['Count'] / trx['Count'].sum(), 3)
    return trx

In [ ]:
df_transaction=pd.read_csv("../data/train_transaction.csv.zip", engine="python")
df_identity=pd.read_csv("../data/train_identity.csv.zip", engine="python")

In [ ]:
df_identity.info()

In [ ]:
df_transaction.info()

In [ ]:
print(df_transaction.shape, df_identity.shape)

In [ ]:
print(df_identity.dtypes.unique(), df_transaction.dtypes.unique())

In [ ]:
df = df_transaction.merge(df_identity, on="TransactionID", how="left")

In [ ]:
df.shape

In [ ]:
df.to_csv("data/df.csv")

In [ ]:
del df_identity, df_transaction, df

In [ ]:
df=pd.read_csv("data/df.csv", engine="python")

In [ ]:
df.columns

In [ ]:
df=df.drop('Unnamed: 0', axis=1)
df.shape

## OPTIMISATION MEMOIRE CODE - DOWNCAST DES COLONNES

In [ ]:
# suppression des colonnes ne contenant que des NA
df=df.dropna(axis=1, how='all')

In [ ]:
# downcast des int et des float
int_columns = df.select_dtypes(include=['int']).columns.tolist()
float_columns = df.select_dtypes(include=['float']).columns.tolist()
df[int_columns] = df[int_columns].apply(pd.to_numeric, downcast='integer')
df[float_columns] = df[float_columns].apply(pd.to_numeric, downcast='float')

In [ ]:
# downcoast des object en category
object_columns=df.select_dtypes(include=['object']).columns.tolist()
df[object_columns]=df[object_columns].apply(lambda x: x.astype('category'))

In [ ]:
df.dtypes.unique()

In [ ]:
PATH_DF = "data/df.csv"

In [ ]:
df_stats = tfdv.generate_statistics_from_csv(data_location=PATH_DF)

In [ ]:
tfdv.visualize_statistics(df_stats)

In [ ]:
df["card4"].value_counts()

In [ ]:
df.shape

## ETUDE BIVARIEE

In [ ]:
# lib 

def missing_values_analysis(df, lst_vars=None):
    if lst_vars is not None:
        missing=df[lst_vars].isnull().sum()
        percent_missing=df[lst_vars].isnull().sum()*100 / len(df)
        missing_value_df = pd.DataFrame({'column_name': df[lst_vars].columns,
                                 'missing': missing,
                                 'percent_missing': percent_missing})
    else:
        missing=df.isnull().sum()
        percent_missing=df.isnull().sum()*100 / len(df)
        missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'missing': missing,
                                 'percent_missing': percent_missing})
    return missing_value_df

def corr_with_Y(df, col):
    vars_quanti=df.select_dtypes(exclude=['object']).columns
    return df[vars_quanti].corrwith(df[col]).reset_index().rename({'index':"Column", 0:'Value'}, axis=1).sort_values("Value", ascending=False)

In [ ]:
# imbalanced of the problem
print(df["isFraud"].value_counts(), df["isFraud"].value_counts(normalize=True))

In [ ]:
vars_quanti=df.select_dtypes(exclude=['object']).columns
vars_quanti

In [ ]:
df[vars_quanti].corrwith(df["isFraud"]).reset_index().rename({'index':"Column", 0:'Value'}, axis=1).sort_values("Value", ascending=False)

In [ ]:
df_fraud=df[df["isFraud"] == 1]
df_fraud.shape

In [ ]:
vars_quanti_fraud=df_fraud.select_dtypes(exclude=['object']).columns

In [ ]:
df[vars_quanti_fraud].corrwith(df_fraud["isFraud"]).reset_index().rename({'index':"Column", 0:'Value'}, axis=1).sort_values("Value", ascending=False)

In [ ]:
df_fraud[vars_quanti_fraud]

In [ ]:
missing_values_analysis(df_fraud, vars_quanti_fraud)

In [ ]:
percent_missing=df_fraud[vars_quanti_fraud].isnull().sum()*100 / len(df_fraud)
missing=df_fraud[vars_quanti_fraud].isnull().sum()
missing_value_df = pd.DataFrame({'column_name': df_fraud[vars_quanti_fraud].columns,
                                 'missing': missing,
                                 'percent_missing': percent_missing})
missing_value_df

In [ ]:
def corr_with_Y(df, col):
    vars_quanti=df.select_dtypes(exclude=['object']).columns
    return df[vars_quanti].corrwith(df[col]).reset_index().rename({'index':"Column", 0:'Value'}, axis=1).sort_values("Value", ascending=False)

In [ ]:
vars_quanti_fraud=df_fraud.select_dtypes(exclude=['object']).columns

In [ ]:
df_fraud[vars_quanti_fraud]

In [ ]:
df_fraud[vars_quanti_fraud].corrwith(df_fraud["isFraud"]).reset_index().rename({'index':"Column", 0:'Value'}, axis=1).sort_values("Value", ascending=False)

In [ ]:
corr_with_Y(df, "isFraud")

### DIFFERENT TYPES OF INFORMATION (COLUMN)

In [ ]:
df

In [ ]:
df.shape

In [ ]:
np.unique(df["ProductCD"])

BROUILLON

In [ ]:
#l = []

#for var in df_identity_quali:
    #print(var)
    #l.append(quali_distribution(df_identity, var))
    
#print(l)

In [ ]:
quali_distribution(df_identity, 'DeviceType')

In [ ]:
df_transaction[df_transaction["isFraud"] == 1].describe()

In [ ]:
df_transaction.describe()

In [ ]:
df_identity.columns

In [ ]:
df_identity.DeviceType.value_counts()

In [ ]:
df_transaction.columns

In [ ]:
df_transaction.isFraud.value_counts(normalize=True)